In [ ]:
#look at README file(mandatory) and requirements.txt first if not using google co lab
#(much preferred to download all of the requirements them just in case)

In [ ]:
!pip install insightface onnxruntime

In [ ]:
from insightface.app import FaceAnalysis

app = FaceAnalysis(name="buffalo_l", providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

In [ ]:
import os
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from torchvision import transforms
from PIL import Image
from google.colab.patches import cv2_imshow

# --------------------------- Configuration -----------------------------------
db_path = 'db'
group_photo_dir = 'group_photos'
supported_exts = ('.jpg', '.jpeg', '.png')
standard_display_width = 800

# ---------------------- Augmentation Transforms ----------------------
augmentation_transforms = [
    transforms.RandomRotation(degrees=20),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ColorJitter(brightness=0.5),
    transforms.ColorJitter(contrast=0.5),
    transforms.ColorJitter(saturation=1.0),
    transforms.ColorJitter(hue=0.3),
    transforms.RandomPerspective(distortion_scale=0.4, p=1),
    transforms.GaussianBlur(kernel_size=(9, 9)),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomGrayscale(p=1),
]

def augment_and_save(img_path, save_dir, base_name):
    image = Image.open(img_path).convert('RGB')
    image.save(os.path.join(save_dir, f"{base_name}_orig.jpg"))
    for i, transform in enumerate(augmentation_transforms):
        augmented = transform(image)
        augmented.save(os.path.join(save_dir, f"{base_name}_aug{i+1}.jpg"))

# ---------------------- Step 1: Organize and Augment DB ----------------------
for file in os.listdir(db_path):
    if file.lower().endswith(supported_exts) and os.path.isfile(os.path.join(db_path, file)):
        actor_name = os.path.splitext(file)[0]
        actor_dir = os.path.join(db_path, actor_name)
        os.makedirs(actor_dir, exist_ok=True)
        img_path = os.path.join(db_path, file)
        augment_and_save(img_path, actor_dir, actor_name)
        os.remove(img_path)

# ---------------------- Step 2: Load DB Embeddings ----------------------

def get_face_embeddings(image):
    faces = app.get(image)
    embeddings, bboxes = [], []
    for face in faces:
        embeddings.append(face.embedding)
        bboxes.append(face.bbox)
    return embeddings, bboxes

db_embeddings, db_names = [], []
actor_dirs = [d for d in os.listdir(db_path) if os.path.isdir(os.path.join(db_path, d))]

for actor in actor_dirs:
    actor_folder = os.path.join(db_path, actor)
    actor_embeddings = []
    for file in os.listdir(actor_folder):
        if file.lower().endswith(supported_exts):
            img_path = os.path.join(actor_folder, file)
            img = cv2.imread(img_path)
            if img is not None:
                emb, _ = get_face_embeddings(img)
                if emb:
                    actor_embeddings.append(emb[0])
    if actor_embeddings:
        avg_emb = np.mean(actor_embeddings, axis=0)
        db_embeddings.append(avg_emb)
        db_names.append(actor)

# ---------------------- Step 3: Face Matching ----------------------
def match_faces(group_embeddings, db_embeddings, threshold=0.35):
    matched_labels, similarity_scores = [], []
    for g_emb in group_embeddings:
        sims = cosine_similarity([g_emb], db_embeddings)[0]
        best_idx = np.argmax(sims)
        best_score = sims[best_idx]
        matched_labels.append(best_idx if best_score > threshold else None)
        similarity_scores.append(best_score)
    return matched_labels, similarity_scores

def annotate_faces(image, bboxes, labels, names, scores):
    predicted_names = []
    y_offsets = {}
    for i, (bbox, label, score) in enumerate(zip(bboxes, labels, scores)):
        if label is None:
            predicted_names.append("Unknown")
            continue
        x1, y1, x2, y2 = map(int, bbox)
        name = names[label]
        predicted_names.append(name)
        color = (0, 255, 0)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Avoid overlapping names
        key = (x1 // 10) * 10
        offset = y_offsets.get(key, 0)
        y_offsets[key] = offset + 20
        text_y = max(10, y1 - 10 + offset)

        cv2.putText(image, f"{name} ({score:.2f})", (x1, text_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)
    return image, predicted_names

# ---------------------- Step 4: Process Group Photos ----------------------
total_TP, total_FP, total_FN, total_TN = 0, 0, 0, 0
precision_list, recall_list, f1_list, acc_list = [], [], [], []

for file in os.listdir(group_photo_dir):
    if file.lower().endswith(supported_exts):
        print(f"\n{'=' * 30}\n📸 Processing: {file}")
        img_path = os.path.join(group_photo_dir, file)
        group_img = cv2.imread(img_path)

        group_embeddings, group_bboxes = get_face_embeddings(group_img)
        matched_indices, scores = match_faces(group_embeddings, db_embeddings)

        annotated_img, predicted_names = annotate_faces(group_img.copy(), group_bboxes, matched_indices, db_names, scores)

        # Resize for consistent visualization
        height, width = annotated_img.shape[:2]
        new_height = int((standard_display_width / width) * height)
        annotated_img_resized = cv2.resize(annotated_img, (standard_display_width, new_height))
        cv2_imshow(annotated_img_resized)

        # Evaluation
        TP, FP, FN, TN = 0, 0, 0, 0
        name_counts = {}

        for name, score in zip(predicted_names, scores):
            if name == "Unknown":
                TN += 1
            else:
                name_counts[name] = name_counts.get(name, 0) + 1
                if name in db_names:
                    TP += 1

        # Count false positives: duplicate names
        FP = sum(count - 1 for count in name_counts.values() if count > 1)

        # Count false negatives for missed faces within similarity range
        for emb in group_embeddings:
            sims = cosine_similarity([emb], db_embeddings)[0]
            best_score = np.max(sims)
            if 0.2 <= best_score < 0.35:
                FN += 1

        total_TP += TP
        total_FP += FP
        total_FN += FN
        total_TN += TN

        precision = TP / (TP + FP + 1e-6)
        recall = TP / (TP + FN + 1e-6)
        f1 = 2 * precision * recall / (precision + recall + 1e-6)
        accuracy = (TP + TN) / (TP + TN + FP + FN + 1e-6)

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        acc_list.append(accuracy)

# ---------------------- Final Averaged Metrics ----------------------
print("\n" + "=" * 40)
print("📊 Final Evaluation Summary Across All Group Photos")
print(f"✅ Total True Positives     : {total_TP}")
print(f"❌ Total False Positives    : {total_FP}")
print(f"🚫 Total False Negatives    : {total_FN} (similarity 0.2–0.34)")
print(f"🟦 Total True Negatives     : {total_TN}")
print("-" * 40)
print(f"📊 Average Accuracy         : {np.mean(acc_list):.2f}")
print(f"📊 Average Precision        : {np.mean(precision_list):.2f}")
print(f"📊 Average Recall           : {np.mean(recall_list):.2f}")
print(f"📊 Average F1 Score         : {np.mean(f1_list):.2f}")
print("=" * 40)


# New Section